# 가중치 가지치기 및 양자화를 통한 모델 축소

## 요약

1. MNIST 데이터를 tf.keras 라이브러리를 이용해 모델링한다.
2. Pruning API를 적용하여 모델을 미세조정하고 정확도를 확인한다.
3. 가지치기에서 3배 더 작은 TF와 TFLite 모델을 만든다.
4. 가지치기와 훈련 후 양자화를 거쳐 10배 더 작은 TFLite 모델을 만든다.
5. 최적화된 TF 및 TFLite 모델의 정확도를 확인한다.

## 라이브러리 불러오기 및 설정

In [1]:
import os
import zipfile
import tempfile

import numpy as np
import tensorflow as tf

import tensorflow_model_optimization as tfmot

In [2]:
# TensorFlow 로그 제어
from freeman.utils.support_tf import LogLevelManager as llm
llm.set(2)      # 경고 이상만 출력(DEBUG, INFO 제외)

In [3]:
BASE_MODEL_DIR = os.path.join(os.path.expanduser("~"), "temp")
FILE_MODEL_NORMAL = os.path.join(BASE_MODEL_DIR, "model_normal.h5")
FILE_MODEL_KERAS = os.path.join(BASE_MODEL_DIR, "model_keras.h5")
FILE_MODEL_TFLITE = os.path.join(BASE_MODEL_DIR, "model_tflite.tflite")

## 일반 모델 훈련(with MNIST)

In [4]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()
train_x.shape, test_x.shape

((60000, 28, 28), (10000, 28, 28))

In [5]:
# 정규화(0~1)
train_x, test_x = train_x / 255., test_x / 255.

In [6]:
model_normal = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28, 28)),
    tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(filters=12, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

model_normal.compile(optimizer="adam",
                     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                     metrics=["accuracy"])

model_normal.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 12)        120       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 12)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2028)              0         
                                                                 
 dense (Dense)               (None, 10)                20290     
                                                                 
Total params: 20,410
Trainable params: 20,410
Non-trainable params: 0
____________________________________________________

In [7]:
%%time
# 모델 훈련
model_normal.fit(train_x, train_y, epochs=5, validation_split=0.1, verbose=0)

CPU times: user 1min 17s, sys: 11.4 s, total: 1min 28s
Wall time: 52.4 s


In [8]:
# 모델 평가
_, accuracy_normal = model_normal.evaluate(test_x, test_y, verbose=0)

In [9]:
# 모델 저장
tf.keras.models.save_model(model_normal, FILE_MODEL_NORMAL, include_optimizer=False)
size_file_normal = os.path.getsize(FILE_MODEL_NORMAL)

## 가지치기(Pruning)를 통한 모델 미세조정

* 가지치기를 전체 모델에 적용하고 모델 요약에서 이를 확인한다.
* 이 예제는 50% 희소성(가중치가 0인 50%)으로 모델을 시작하고, 80% 희소성으로 종료한다.
* <b>모델 정확도를 높이기 위해 일부 레이어를 잘라낼 수 도 있다.</b>(이 예제에는 없음)

In [10]:
# Pruning 옵션 설정

## 여기서 정의된 batch_size, epochs 등은 위에서 학습된 모델에서 사용된 것이 아니라,
## 가지치기를 위한 파라미터임.
PRUNE_BATCH_SIZE = 128
PRUNE_EPOCHS = 5
PRUNE_VALIDATION_SPLIT = 0.1

PRUNE_TRAIN_DATA_SIZE = train_x.shape[0] * (1 - PRUNE_VALIDATION_SPLIT)
PRUNE_END_STEP = np.ceil(PRUNE_TRAIN_DATA_SIZE/PRUNE_BATCH_SIZE).astype(np.int32) * PRUNE_EPOCHS

In [11]:
pruning_params = {
    "pruning_schedule":
        tfmot.sparsity.keras.PolynomialDecay(
            initial_sparsity=0.50,
            final_sparsity=0.80,
            begin_step=0,
            end_step=PRUNE_END_STEP
        ),
}

# 일반 모델에 가지치기를 한 모델 정의
model_keras = tfmot.sparsity.keras.prune_low_magnitude(model_normal, **pruning_params)
# 가지치기 모델 컴파일(일반 모델과 동일(컴파일/훈련/평가)하게 처리됨)
model_keras.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [15]:
%%time

logdir = tempfile.mkdtemp()

# 가지치기 모델 훈련용 콜백함수 정의
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),               # 훈련중 사용
    tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),  # 진행사항 추적 및 디버깅용 로그 제공
]

# 가지치기 모델 훈련
model_keras.fit(
    train_x, train_y,
    batch_size=PRUNE_BATCH_SIZE,
    epochs=PRUNE_EPOCHS,
    validation_split=PRUNE_VALIDATION_SPLIT,
    callbacks=callbacks,
    verbose=0
)

CPU times: user 42.3 s, sys: 7.71 s, total: 50.1 s
Wall time: 20.5 s


In [16]:
!tensorboard --logdir={logdir}


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.10.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [17]:
_, accuracy_keras = model_keras.evaluate(test_x, test_y, verbose=0)

In [23]:
tf.keras.models.save_model(model_keras, FILE_MODEL_KERAS, include_optimizer=False)

## 일반 및 가지치기 모델 비교

In [12]:
model_normal.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 12)        120       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 12)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2028)              0         
                                                                 
 dense (Dense)               (None, 10)                20290     
                                                                 
Total params: 20,410
Trainable params: 20,410
Non-trainable params: 0
____________________________________________________

In [13]:
model_keras.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshape  (None, 28, 28, 1)        1         
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_conv2d   (None, 26, 26, 12)       230       
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_max_poo  (None, 13, 13, 12)       1         
 ling2d (PruneLowMagnitude)                                      
                                                                 
 prune_low_magnitude_flatten  (None, 2028)             1         
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense (  (None, 10)               4

## 모델 축소

### 압축 가능한 모델로 변경

In [18]:
# 학습 완료된 가지치기 모델을 압축 가능한 모델로 변경
model_keras_before_zip = tfmot.sparsity.keras.strip_pruning(model_keras)

# 압축 가능한 모델을 이용해 TFLite 모델로 생성
tflite_converter = tf.lite.TFLiteConverter.from_keras_model(model_keras_before_zip)
model_tflite_before_zip = tflite_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp5xo4b3wd/assets


In [19]:
# 압축 가능한 모델 저장
_, temp_keras_file = tempfile.mkstemp(".h5")
_, temp_tflite_file = tempfile.mkstemp(".tflite")

tf.keras.models.save_model(model_keras_before_zip, temp_keras_file, include_optimizer=False)
with open(temp_tflite_file, "wb") as f:
    f.write(model_tflite_before_zip)

### 모델 압축

In [22]:
# 모델 압축함수 정의
def gzipped_model(file):
    _, zipped_file = tempfile.mkstemp(".zip")
    with zipfile.ZipFile(zipped_file, "w", compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)
    return os.path.getsize(zipped_file)

In [25]:
size_file_keras = os.path.getsize(FILE_MODEL_KERAS)
size_file_keras_zip = gzipped_model(temp_keras_file)
size_file_tflite = os.path.getsize(temp_tflite_file)
size_file_tflite_zip = gzipped_model(temp_tflite_file)

## 가지치기와 양자화를 결합해 10배 더 작은 모델 생성

In [28]:
tflite_converter = tf.lite.TFLiteConverter.from_keras_model(model_keras_before_zip)
tflite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
model_quantized = tflite_converter.convert()

_, temp_quantized_file = tempfile.mkstemp(".tflite")
with open(temp_quantized_file, "wb") as f:
    f.write(model_quantized)
    
size_file_quantized = os.path.getsize(temp_quantized_file)
size_file_quantized_zip = gzipped_model(temp_quantized_file)

INFO:tensorflow:Assets written to: /tmp/tmpvpwir19a/assets


INFO:tensorflow:Assets written to: /tmp/tmpvpwir19a/assets


## 압축파일 정확도 확인

In [29]:
def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]
    
    prediction_digits = []
    for i, test_data in enumerate(test_x):
        if i % 2000 == 0:
            print(".", end="")
        test_data = np.expand_dims(test_data, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_data)
        interpreter.invoke()
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)
    print()
    
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == test_y).mean()
    return accuracy

In [30]:
interpreter = tf.lite.Interpreter(model_content=model_quantized)
interpreter.allocate_tensors()
accuracy_tflite = evaluate_model(interpreter)

.....
